## Amharic Named Entity Recognition (NER) 

In [ ]:
# !pip install seqeval
# !pip install transformers
# !pip install datasets

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

### Importing the necessary libraries

In [ ]:
sys.path.append(os.path.abspath('../scripts'))

from tunning import Tunning, Prepocess

#### Loading conll format using datasets

In [3]:
filepath= '..data/conll_output.conll'

preprocesss = Prepocess()
data = preprocesss.read_conll_file(filepath)
datasets = preprocesss.process(filepath)
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 52304
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13076
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 16346
    })
})

## Afroxlmr

In [ ]:
label_list = sorted(list(set([token_data[1] for sentence in data for token_data in sentence])))
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

In [ ]:
fine_tune = Tunning()
fine_tune.tokenize_train_args(datasets, epochs=3, eval_strategy='epoch')
trainer = fine_tune.train(tokenizer, model)

In [7]:
train_output = trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.514500,0.459560,0.747649,0.532397,0.621925
2,0.421100,0.427247,0.802680,0.576603,0.671113
3,0.387600,0.408725,0.816852,0.601211,0.692635


In [8]:
train_output

TrainOutput(global_step=19614, training_loss=0.4410831937248266, metrics={'train_runtime': 1900.3207, 'train_samples_per_second': 82.571, 'train_steps_per_second': 10.321, 'total_flos': 2791699357787760.0, 'train_loss': 0.4410831937248266, 'epoch': 3.0})

In [9]:
trainer.save_model('distilbert_base_multilingual_model')
tokenizer.save_pretrained('distilbert_base_multilingual_tokenizer')

('distilbert_base_multilingual_tokenizer/tokenizer_config.json',
 'distilbert_base_multilingual_tokenizer/special_tokens_map.json',
 'distilbert_base_multilingual_tokenizer/vocab.txt',
 'distilbert_base_multilingual_tokenizer/added_tokens.json',
 'distilbert_base_multilingual_tokenizer/tokenizer.json')